In [ ]:

##################################
#                      NEW MODEL
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt 


maydf=pd.read_csv('/content/drive/My Drive/MayFinal.csv')
maydf.dropna(inplace=True)

dataN=maydf[['Ref.L.Cell.Thrp.Kbps.DL(kbit/s)','Ref.Avg.CQI(number)','Ref.DL.PRB.utilization(%)','L.UL.Interference.Avg(dBm)','Ref.L.RA.TA.UE.Index_All_Approx','Ish.L.User.Avg(number)']]



# convert column names - avoid problemns in column indexing
dataN.rename(columns={"Ref.L.Cell.Thrp.Kbps.DL(kbit/s)":"Cell.Thrp.Kbps.DL"},inplace=True)
dataN.columns=dataN.columns.astype(str).str.replace("[.]", "_")
dataN.columns=dataN.columns.astype(str).str.replace("[(]", "_")
dataN.columns=dataN.columns.astype(str).str.replace("[)]", "_")
dataN.columns=dataN.columns.astype(str).str.replace("[%]", "X")
print(dataN.columns)

# drop rows with cell trpt =0
dataN=dataN[dataN.loc[:,"Cell_Thrp_Kbps_DL"]!=0]

# create classes 
dataN['CellT_category']=pd.cut(dataN['Cell_Thrp_Kbps_DL'], bins=np.arange(0, 70001, 10000),labels=np.arange(0,70000,10000).tolist())

# set x and y variables
y=dataN['CellT_category']

X=dataN[['Ref_Avg_CQI_number_', 'Ref_DL_PRB_utilization_X_',
       'L_UL_Interference_Avg_dBm_', 'Ref_L_RA_TA_UE_Index_All_Approx',
       'Ish_L_User_Avg_number_']]

# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =  train_test_split(X,y,test_size = 0.3,random_state=10)





In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# use random grid to tune the model

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

# create random forest classifier
rf = RandomForestClassifier()

# apply grid
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 15, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model with train set
rf_random.fit(X_train, y_train)


Fitting 3 folds for each of 15 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 88.3min
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed: 121.0min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [ ]:
rf_random.best_params_ # get best parameters

{'bootstrap': False,
 'max_depth': 80,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'n_estimators': 1000}

In [ ]:
best_rfc = rf_random.best_estimator_

brscore=best_rfc.score(X_test, y_test) # accuracy on test data
rfc_predict = best_rfc.predict(X_test) # predict test data
brscore

0.9573417274972186

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
# confusion metrix  the diagol is number of correct predictions
cm=confusion_matrix(y_test,rfc_predict)
cm

array([[ 3847,   477,     0,     0,     0,     0],
       [  194, 27125,   295,     0,     0,     0],
       [    8,   563,  4205,     4,     0,     0],
       [    0,    13,    16,    94,     0,     0],
       [    0,     0,     0,     0,     7,     0],
       [    0,     2,     0,     0,     0,     1]])

In [ ]:
import pickle
# save model
filename='rfcmoel.sav'
pickle.dump(best_rfc, open(filename, 'wb'))

NameError: ignored

In [ ]:
import pickle
#load model
loaded_model = pickle.load(open('/content/drive/My Drive/rfcmodelZip.zip (Unzipped Files)/rfcmodel.sav', 'rb'))

In [ ]:
loaded_model

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=80, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:


#####################
#########################
# draw any estimator (a tree from collection of trees)  

feature_cols =['Ref_Avg_CQI_number_','Ref_DL_PRB_utilization_X_','L_UL_Interference_Avg_dBm_','Ref_L_RA_TA_UE_Index_All_Approx','Ish_L_User_Avg_number_']
classes=np.arange(0,70001,10000).tolist()
classesX=[str(item) for item in classes]
classesX



estimator = loaded_model.estimators_[1] # estimator id


from sklearn.tree import export_graphviz


export_graphviz(estimator, out_file='tree.dot', 
                feature_names = feature_cols,
                class_names = classesX,
                rounded = True, proportion = False, 
                filled = True,max_depth=15)



from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=3000'])

# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')

Output hidden; open in https://colab.research.google.com to view.